# Predict Future Sales (Kaggle playground)

I accepted the terms & conditions then downloaded the data.
`kaggle competitions download -c competitive-data-science-predict-future-sales`

'Prelude' copied from lesson3-rossmann:

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
import functools
from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)
from IPython.display import HTML

PATH='data/predict-future-sales/'

In [2]:
!ls {PATH}

item_categories.csv  sales_train.csv	       shops.csv
items.csv	     sample_submission.csv.gz  test.csv.gz


## Import data

In [3]:
table_names = ['item_categories', 'items', 'shops', 'sales_train', 'test']

def load_table(table, root_path):
    fname = os.path.join(root_path, f"{table}.csv")
    if os.path.exists(fname):
        compression = None
    else:
        fname = f"{fname}.gz"
        compression = 'gzip'
    
    return pd.read_csv(fname, compression=compression)

tables = [load_table(t, root_path=PATH) for t in table_names]

In [4]:
for t in tables: display(t.head())

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [5]:
for t in tables:
    display(DataFrameSummary(t).summary())

,item_category_name,item_category_id
count,NaN,84
mean,NaN,41.5
std,NaN,24.3926
min,NaN,0
25%,NaN,20.75
50%,NaN,41.5
75%,NaN,62.25
max,NaN,83
counts,84,84
uniques,84,84


,item_name,item_id,item_category_id
count,NaN,22170,22170
mean,NaN,11084.5,46.2908
std,NaN,6400.07,15.9415
min,NaN,0,0
25%,NaN,5542.25,37
50%,NaN,11084.5,40
75%,NaN,16626.8,58
max,NaN,22169,83
counts,22170,22170,22170
uniques,22170,22170,84


,shop_name,shop_id
count,NaN,60
mean,NaN,29.5
std,NaN,17.4642
min,NaN,0
25%,NaN,14.75
50%,NaN,29.5
75%,NaN,44.25
max,NaN,59
counts,60,60
uniques,60,60


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,NaN,2.93585e+06,2.93585e+06,2.93585e+06,2.93585e+06,2.93585e+06
mean,NaN,14.5699,33.0017,10197.2,890.853,1.24264
std,NaN,9.42299,16.227,6324.3,1729.8,2.61883
min,NaN,0,0,0,-1,-22
25%,NaN,7,22,4476,249,1
50%,NaN,14,31,9343,399,1
75%,NaN,23,47,15684,999,1
max,NaN,33,59,22169,307980,2169
counts,2935849,2935849,2935849,2935849,2935849,2935849
uniques,1034,34,60,21807,19993,198


,ID,shop_id,item_id
count,214200,214200,214200
mean,107100,31.6429,11019.4
std,61834.4,17.5619,6252.64
min,0,2,30
25%,53549.8,16,5381.5
50%,107100,34.5,11203
75%,160649,47,16071.5
max,214199,59,22167
counts,214200,214200,214200
uniques,214200,42,5100


In [6]:
item_categories, items, shops, sales_train, test = tables
sales_train = sales_train.copy()
sales_train['gross'] = sales_train['item_cnt_day'] * sales_train['item_price']

def reformat_ddmmyyyy_to_yyyymmdd(x):
    return "{2}-{1}-{0}".format(*(x.split('.')))

sales_train['date'] = sales_train['date'].apply(reformat_ddmmyyyy_to_yyyymmdd)
add_datepart(sales_train, 'date', drop=False)

Copied from lesson3-rossmann:

`join_df` is a function for joining tables on specific fields. By default, we'll be doing a left outer join (i.e. inner join + keep rows of left table that don't match anything in the right table) of right on the left argument using the given fields for each table.

Pandas does joins using the merge method. The suffixes argument describes the naming convention for duplicate fields. We've elected to leave the duplicate field names on the left untouched, and append a `"_y"` to those on the right.


In [7]:
def join_df(left, right, left_on, right_on=None, suffix='_y'):
    """@param left: Dataframe
    @param right: Dataframe
    @param left_on: column name in left table
    @param right_on: (default: left_on) column name in right table
    @param suffix: (default: "_y") appended to duplicate column names from the right table"""
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                      suffixes=("", suffix))

def denormalize(t):
    """Denormalize table by adding shop names, item names & item categories
    
    This function is specific to the data model of this Kaggle competition."""
    t = join_df(t, shops, 'shop_id')
    t = join_df(t, items, 'item_id')
    t = join_df(t, item_categories, 'item_category_id')
    return t

### Aggregate training data per month.

First, enrich the sales data with all other tables we have, so that we can e.g. group by categories instead of items later.

In [8]:
sales_train = denormalize(sales_train)

Looking at the data, it seemed like at least one shop was open every day, but there are shops that were not open every day.  Record per shop how many days they were open per month, how many items (all lumped together) they sold and what revenue they generated.

In [9]:
open_days_sales_and_monthly_revenue = sales_train.groupby(['date_block_num', 'shop_id'], as_index=False).agg({
    'date': lambda x: len(pd.unique(x)),
    'item_cnt_day': 'sum',
    'gross': 'sum'})
open_days_sales_and_monthly_revenue.rename(inplace=True, columns={
    'date': 'open_days',
    'item_cnt_day': 'all_shop_items_sold',
    'gross': 'all_shop_gross'
});

How many items of each sort were sold in all stores for each time period and what revenue did each item generate?

In [10]:
items_globally = sales_train.groupby(['date_block_num', 'item_id'], as_index=False).agg({
    'item_cnt_day': 'sum',
    'gross': 'sum'})
items_globally.rename(inplace=True, columns={
    'item_cnt_day': 'global_sold',
    'gross': 'global_gross'
});

How many items of each sort were sold per store for each time period and what revenue did they generate?

In [16]:
items_per_shop = sales_train.groupby(['date_block_num', 'shop_id', 'item_id'], as_index=False).agg({
    'item_cnt_day': 'sum',
    'Year': 'min', # constant inside a grouping anyway
    'Month': 'min', # constant inside a grouping anyway
    'item_price': 'mean', # actually I hope it's constant inside a month...
    'gross': 'sum'})

How much (count and revenue) was sold per category for each time period (globally & per store)?

In [12]:
cats_globally = sales_train.groupby(['date_block_num', 'item_category_id'], as_index=False).agg({
    'item_cnt_day': 'sum',
    'gross': 'sum'})
cats_globally.rename(inplace=True, columns={
    'item_cnt_day': 'global_cat_sold',
    'gross': 'global_cat_gross'
});
cats_per_shop = sales_train.groupby(['date_block_num', 'shop_id', 'item_category_id'], as_index=False).agg({
    'item_cnt_day': 'sum',
    'gross': 'sum'})
cats_per_shop.rename(inplace=True, columns={
    'item_cnt_day': 'cat_shop_sold',
    'gross': 'cat_shop_gross'
});

Paste together all the cumulative data, the goal is a table with unique (date_block_num, shop_id, item_id) rows.  Start from `items_per_shop` then join other cumulative data onto it.

In [17]:
 data = functools.reduce(
    lambda src, extra: join_df(src, *extra),
    [[items, 'item_id'],
     [item_categories, 'item_category_id'],
     [shops, 'shop_id'],
     [open_days_sales_and_monthly_revenue, ('date_block_num', 'shop_id')],
     [items_globally, ('date_block_num', 'item_id')],
     [cats_globally, ('date_block_num', 'item_category_id')],
     [cats_per_shop, ('date_block_num', 'shop_id', 'item_category_id')]],
    items_per_shop)

In [18]:
data.columns

Index(['date_block_num', 'shop_id', 'item_id', 'item_cnt_day', 'Year', 'Month',
       'item_price', 'gross', 'item_name', 'item_category_id',
       'item_category_name', 'shop_name', 'open_days', 'all_shop_items_sold',
       'all_shop_gross', 'global_sold', 'global_gross', 'global_cat_sold',
       'global_cat_gross', 'cat_shop_sold', 'cat_shop_gross'],
      dtype='object')

In [23]:
cat_vars = ['shop_id', 'item_id', 'item_name', 'item_category_id', 'item_category_name', 'shop_name', 'Year', 'Month']
cont_vars = ['item_price',
       'gross', 'open_days', 'all_shop_items_sold', 'all_shop_gross',
       'global_sold', 'global_gross', 'global_cat_sold', 'global_cat_gross',
       'cat_shop_sold', 'cat_shop_gross']

In [24]:
len(data.columns) - len(cat_vars) - len(cont_vars) - 2 # 2: dependent variable & time

0

Reorder data & cast to types expected by PyTorch (float32 for continuous variables, explicitly label categorical variables as such) (copied from lesson3-rossmann).

In [25]:
dep = 'item_cnt_day'

for v in cat_vars: 
    data[v] = data[v].astype('category').cat.as_ordered()

for v in cont_vars:
    data[v] = data[v].astype('float32')

data = data[cat_vars+cont_vars+[dep, 'date_block_num']].copy()

Todo:

  1. Embedding for item name & item category name
  2. Skip shop name?
  3. Drop 'gross' column: it is too correlated with item_cnt_day and I don't have it in the test set
  4. Need to apply same enrichment to test data (i.e. add item categories)
  5. time series yet only data from the same month is used?  I don't even have it for the test data!  Incorporate results of previous months, previous months results
  6. Define fitness function, clip output to [0, 20]

Select subset of data to speed up exploration

In [27]:
n = data.shape[0]

In [29]:
idxs = get_cv_idxs(n, val_pct=150000/n) 
joined_samp = data.iloc[idxs].set_index("date_block_num") 
samp_size = len(joined_samp); samp_size

150000

In [31]:
joined_samp.head(2)

,shop_id,item_id,item_name,item_category_id,item_category_name,shop_name,Year,Month,item_price,gross,open_days,all_shop_items_sold,all_shop_gross,global_sold,global_gross,global_cat_sold,global_cat_gross,cat_shop_sold,cat_shop_gross,item_cnt_day
date_block_num,,,,,,,,,,,,,,,,,,,,
17,38,2196,COLDPLAY Ghost Stories,55,Музыка - CD локального производства,"Омск ТЦ ""Мега""",2014,6,299.0,598.0,30.0,1467.0,1533191.0,145.0,43012.449219,10769.0,3077388.25,126.0,35179.00000,2.0
19,31,9964,ВОЗДУШНЫЙ МАРШАЛ,40,Кино - DVD,"Москва ТЦ ""Семеновский""",2014,8,399.0,2793.0,31.0,8248.0,5763062.5,93.0,36641.898438,15821.0,4307263.50,1841.0,474910.09375,7.0


In [36]:
df, y, nas, mapper = proc_df(joined_samp, dep, do_scale=True, skip_flds=['gross'])
yl = np.log(y)

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log
  


In [37]:
sum(y == 0.0)

256

In [38]:
df.head(2)

,shop_id,item_id,item_name,item_category_id,item_category_name,shop_name,Year,Month,item_price,open_days,all_shop_items_sold,all_shop_gross,global_sold,global_gross,global_cat_sold,global_cat_gross,cat_shop_sold,cat_shop_gross
date_block_num,,,,,,,,,,,,,,,,,,
17,39,2152,2152,56,56,39,2,6,-0.324435,-0.054396,-0.721803,-0.610916,0.375586,-0.044211,0.168194,-0.316104,-0.401182,-0.542709
19,32,9776,9776,41,41,32,2,8,-0.258757,0.472730,1.852293,1.225618,0.149892,-0.058190,0.749037,-0.052203,3.097769,1.584466


In [39]:
val_idx = np.flatnonzero(df.index == max(df.index))

In [40]:
val_idx.shape

(2956,)

In [41]:
n

1609124